### Setting up Kaggle API in Google Colaboratory

In [0]:
!pip install -q kaggle

In [0]:
f = open("kaggle.json", "w")
f.write('{"username":"pranjalya","key":"fa4deb4f89e2fe282a3e29c166413ecf"}')
f.close()

In [0]:
!mkdir ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d skylord/coronawhy

100% 13.2G/13.2G [03:36<00:00, 66.7MB/s]
100% 13.2G/13.2G [03:36<00:00, 65.4MB/s]


## Finding papers containing pollution keywords in correlation with COVID-19 lethal outcomes

**Task at hand** :  https://trello.com/c/SA27hbNo/35-finding-papers-on-correlation-between-pollution-and-lethal-outcomes

In [7]:
!pip install spacy_langdetect

     |████████████████████████████████| 1.0MB 9.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993459 sha256=cf65fcc7e4257914f15b8cbab856a650be2956773d667bba8fcc6fbc092d49e6
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [8]:
!unzip coronawhy.zip v6_text/*
!unzip coronawhy.zip clean_metadata.csv

Archive:  coronawhy.zip
  inflating: v6_text/v6_text/v6_text_0.pkl  
  inflating: v6_text/v6_text/v6_text_1.pkl  
  inflating: v6_text/v6_text/v6_text_10.pkl  
  inflating: v6_text/v6_text/v6_text_11.pkl  
  inflating: v6_text/v6_text/v6_text_12.pkl  
  inflating: v6_text/v6_text/v6_text_13.pkl  
  inflating: v6_text/v6_text/v6_text_14.pkl  
  inflating: v6_text/v6_text/v6_text_15.pkl  
  inflating: v6_text/v6_text/v6_text_16.pkl  
  inflating: v6_text/v6_text/v6_text_17.pkl  
  inflating: v6_text/v6_text/v6_text_18.pkl  
  inflating: v6_text/v6_text/v6_text_19.pkl  
  inflating: v6_text/v6_text/v6_text_2.pkl  
  inflating: v6_text/v6_text/v6_text_3.pkl  
  inflating: v6_text/v6_text/v6_text_4.pkl  
  inflating: v6_text/v6_text/v6_text_5.pkl  
  inflating: v6_text/v6_text/v6_text_6.pkl  
  inflating: v6_text/v6_text/v6_text_7.pkl  
  inflating: v6_text/v6_text/v6_text_8.pkl  
  inflating: v6_text/v6_text/v6_text_9.pkl  
Archive:  coronawhy.zip
  inflating: clean_metadata.csv      


In [0]:
# Importing required libraries

import os
import pandas as pd
import numpy as np

PATH = "v6_text/v6_text/"

Here are the keywords (currently, we have trigrams) to search for in the papers.

In [0]:
# Pollution n-grams

pollution = ['air pollution and', 'indoor air pollutants', 'indoor air pollution', 'household air pollution', 
             'air pollution is', 'between air pollution', 'of air pollution', 'particulate air pollution',
             'pollution and the', 'air pollutant data']

The output is supposed to be in this format.

In [0]:
df_output = pd.DataFrame(columns=['Risk Factor', 'Keyword/Ngram', 'Paper ID', 'URL', 
                                  'Sentences from Result', 'Sentences from Method', 
                                  'Design Methodology', 'Correlation', 'Authors', 
                                  'No of Citations', 'No of keyword occurence in Paper'])

**Searching for the keywords only in method and result section of the papers, as the most relevant papers can be found there only.**

(This cell may take some time)

In [0]:
firstpass = True

for pkl in os.listdir(PATH):
  df = pd.read_pickle(PATH+pkl, compression='gzip')
  if(firstpass):
    df_method = df[df['section']=='methods']
    df_result = df[df['section']=='results']
    firstpass = False
  else:
    df_method = pd.concat([df_method, df[df['section']=='methods']])
    df_result = pd.concat([df_result, df[df['section']=='results']])

In [13]:
print("No of unique papers in method section : ", df_method['paper_id'].nunique(), " out of ", 
      len(df_method), " rows in dataframe")
print("No of unique papers in result section : ", df_result['paper_id'].nunique(), " out of ", 
      len(df_result), " rows in dataframe")

No of unique papers in method section :  9692  out of  187090  rows in dataframe
No of unique papers in result section :  7820  out of  178050  rows in dataframe


In [14]:
df_method.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187090 entries, 288 to 370559
Data columns (total 36 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   paper_id                         187090 non-null  string
 1   language                         187090 non-null  object
 2   section                          187090 non-null  object
 3   sentence                         187090 non-null  string
 4   lemma                            187090 non-null  object
 5   UMLS                             187090 non-null  object
 6   GGP                              187090 non-null  object
 7   SO                               187090 non-null  object
 8   TAXON                            187090 non-null  object
 9   CHEBI                            187090 non-null  object
 10  GO                               187090 non-null  object
 11  CL                               187090 non-null  object
 12  DNA           

*Just some function I thought I would use, but never did, actually. Maybe someone could find them useful.

In [0]:
def isPresentInList(A, B):
  '''
  A, B : Lists
  Returns : Boolean value for whether A appears as sequence in B
  '''
  return ', '.join(map(str, A)) in ', '.join(map(str, B))

***Now we extract sentences from the method and result sections of papers, which contain the keywords.***

In [0]:
df_method_p = df_method[df_method['sentence'].str.contains('|'.join(pollution), case=False) == True]
df_result_p = df_result[df_result['sentence'].str.contains('|'.join(pollution), case=False) == True]

# Merging the method and result section sentences into single dataframe
df_real = pd.concat([df_method_p, df_result_p])

In [17]:
print("There are {} sentences containing keywords/ngrams in Method section.".format(len(df_method_p)))
print("There are {} sentences containing keywords/ngrams in Result section.".format(len(df_result_p)))

There are 38 sentences containing keywords/ngrams in Method section.
There are 8 sentences containing keywords/ngrams in Result section.


In [18]:
print("Total unique papers in Method section : {}".format(df_method_p['paper_id'].nunique()))
print("Total unique papers in Result section : {}".format(df_result_p['paper_id'].nunique()))
print("Total unique papers in combined section : {}".format(df_real['paper_id'].nunique()))

Total unique papers in Method section : 17
Total unique papers in Result section : 6
Total unique papers in combined section : 20


Hence, we have 20 papers which mention mention the specified keywords.

In [19]:
metadata = pd.read_csv('clean_metadata.csv')
metadata.rename(columns={'sha':'paper_id'}, inplace = True)
metadata['paper_id'] = metadata['paper_id'].astype("string")
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31745 entries, 0 to 31744
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   31745 non-null  int64  
 1   cord_uid                     31745 non-null  object 
 2   paper_id                     31744 non-null  string 
 3   source_x                     31745 non-null  object 
 4   title                        31710 non-null  object 
 5   doi                          31745 non-null  object 
 6   pmcid                        16636 non-null  object 
 7   pubmed_id                    24739 non-null  float64
 8   license                      31745 non-null  object 
 9   abstract                     27917 non-null  object 
 10  publish_time                 31745 non-null  object 
 11  authors                      31191 non-null  object 
 12  journal                      30623 non-null  object 
 13  Microsoft Academ

In [20]:
df_real.head(2)

,paper_id,language,section,sentence,lemma,UMLS,GGP,SO,TAXON,CHEBI,GO,CL,DNA,CELL_TYPE,CELL_LINE,RNA,PROTEIN,DISEASE,CHEMICAL,CANCER,ORGAN,TISSUE,ORGANISM,CELL,AMINO_ACID,GENE_OR_GENE_PRODUCT,SIMPLE_CHEMICAL,ANATOMICAL_SYSTEM,IMMATERIAL_ANATOMICAL_ENTITY,MULTI-TISSUE_STRUCTURE,DEVELOPING_ANATOMICAL_STRUCTURE,ORGANISM_SUBDIVISION,CELLULAR_COMPONENT,PATHOLOGICAL_FORMATION,ORGANISM_SUBSTANCE,sentence_id
119389,142a615ffb970d12beaa9597bff2b9c49da4bb96,en,methods,We searched a variety of databases-Medline (Ov...,"[-PRON-, search, a, variety, of, databases-med...","[ovid, Global Health, Combinations (Undergarme...",[],[],[],[],[],[],[],[],[],[],[],[pneumonia],"[databases-Medline, smoking]",[],[breast],[],[],[],[],[],"[databases-Medline, Ovid]",[],[],[],[],[],[],[],[],142a6551934bb96
238396,7bc929ab19e608e85e901dd0e3fe014ec8761a60,en,methods,The systematic approach involves the measureme...,"[the, systematic, approach, involve, the, meas...","[Systems Analysis, Measurement, Clinical Trial...",[],[],[],[pollutants],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],7bc922187361a60


In [21]:
metadata.head(1)

,Unnamed: 0,cord_uid,paper_id,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,http://doi.org/10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727


Merging the given papers with their metadata, which contains relevant data.

In [0]:
df_real = df_real.merge(metadata[['paper_id', 'title', 'abstract', 'publish_time', 'authors', 'url']], on='paper_id', how='left')

Keeping only the fields which are relevant to us.

In [23]:
df_real = df_real[['paper_id', 'language', 'section', 'sentence', 'lemma', 'UMLS', 'sentence_id', 'title', 'abstract', 'publish_time', 'authors', 'url']]
df_real.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   paper_id      46 non-null     string
 1   language      46 non-null     object
 2   section       46 non-null     object
 3   sentence      46 non-null     string
 4   lemma         46 non-null     object
 5   UMLS          46 non-null     object
 6   sentence_id   46 non-null     string
 7   title         45 non-null     object
 8   abstract      45 non-null     object
 9   publish_time  45 non-null     object
 10  authors       42 non-null     object
 11  url           45 non-null     object
dtypes: object(9), string(3)
memory usage: 4.7+ KB


**TO WORK ON : ** If we need to also get all the papers which show lethality, among the papers containing risk factor keywords.

In [0]:
lethal = ["death", "fatality", "mortality","lethal", "lethality", "morbidity"]

In [0]:
def keywordcounter(sentences):
  '''
  Input : List of sentences
  Returns : Keywords present in sentences, Total count of all keywords present in Input
  '''
  keyword = {}
  sent = " ".join(sentences)
  for pol in pollution:
    counter = sent.lower().count(pol)
    if (counter > 0):
      keyword[pol] = counter
  return list(keyword.keys()), sum(keyword.values())

***Grouping the sentences by their Paper IDs.***

In [0]:
grouped = df_real.groupby('paper_id')

Function to format all the code in specified output format.

Work left : 

-> No of citations for each paper

-> Correlation (Unclear)

-> Design Methodology (Unclear)

In [0]:
def aggregation(item):
  '''
  Input : Dataframe of sentences of a paper
  Return : Datframe in Standard Output format
  '''
  dfo = {}

  dfo['Risk Factor'] = 'Pollution'
  dfo['Keyword/Ngram'], dfo['No of keyword occurence in Paper'] = keywordcounter(item['sentence'].tolist())
  dfo['Paper ID'] = item['paper_id'].iloc[0]
  dfo['URL'] = item['url'].iloc[0]
  dfo['Sentences from Method'] = item[item['section']=='methods']['sentence'].tolist()
  dfo['Sentences from Result'] = item[item['section']=='results']['sentence'].tolist()
  dfo['Authors'] = item['authors'].iloc[0]
  dfo['No of Citations'] = 0
  dfo['Correlation'] = 'Not yet'
  dfo['Design Methodology'] = 'None'

  return dfo

Applying function to get the final output dataframe.

In [0]:
for key, item in grouped:
  df_output = pd.concat([df_output, pd.DataFrame([aggregation(item)])])

In [29]:
df_output = df_output.reset_index()
del df_output['index']
df_output

,Risk Factor,Keyword/Ngram,Paper ID,URL,Sentences from Result,Sentences from Method,Design Methodology,Correlation,Authors,No of Citations,No of keyword occurence in Paper
0,Pollution,[indoor air pollution],045b111f0f2584890e9271399aa93c917a496662,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,[],[Some patients participated in linked case-con...,None,Not yet,"Aston, Stephen J.; Ho, Antonia; Jary, Hannah; ...",0,1
1,Pollution,[air pollution and],0ce2f11a7c4da992b9a8686bd3b2ed53064a2ac8,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,[],[Hourly data for air pollution and meteorologi...,None,Not yet,"Chen, Pei-Shih; Tsai, Feng Ta; Lin, Chien Kun;...",0,1
2,Pollution,"[indoor air pollution, of air pollution]",13a1c758877b216e51c51e9ee532ab5c3e0c12fb,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,[],[Although ecologic fallacy and uncontrolled co...,None,Not yet,"Xia, Tian; Zhu, Yifang; Mu, Lina; Zhang, Zuo-F...",0,2
3,Pollution,[indoor air pollution],142a615ffb970d12beaa9597bff2b9c49da4bb96,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,"[Of these, 7 risk factors were significantly a...",[We searched a variety of databases-Medline (O...,None,Not yet,"Jackson, Stewart; Mathews, Kyle H.; Pulanić, D...",0,3
4,Pollution,[indoor air pollutants],142ecef30a98e08395505ea44e0a7a6f8239ed6b,https://doi.org/10.1016/j.cej.2010.11.061,[Indoor air pollutants consist of particulates...,[],None,Not yet,"Yao, Nan; Lun Yeung, King",0,1
5,Pollution,[of air pollution],21d98e3c9427e96a21a4dc7f37d420c42cd6e864,https://doi.org/10.1016/j.envres.2010.05.005,[],[The analytic method used in this study follow...,None,Not yet,"Thach, Thuan-Quoc; Wong, Chit-Ming; Chan, King...",0,1
6,Pollution,"[indoor air pollutants, indoor air pollution]",283e32b6190b8c6183e0f5cbfae20c01d7df035c,https://doi.org/10.1016/j.rppneu.2014.02.006,[],[Since the first studies on indoor air quality...,None,Not yet,"Araújo-Martins, J.; Carreiro Martins, P.; Vieg...",0,4
7,Pollution,"[air pollution and, between air pollution]",32412ffe52eb41da8ecfed314b703c49fd041f5c,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,[To study the possible relationship between ai...,[],None,Not yet,"Almagro, Pere; Hernandez, Carme; Martinez-Camb...",0,2
8,Pollution,[household air pollution],5ec597fac768d34f7c66553b0d568a5c6f3e2aa0,https://doi.org/10.1016/s1473-3099(19)30410-4,[],[Although decreased exposure to household air ...,None,Not yet,"Troeger, Christopher E; Khalil, Ibrahim A; Bla...",0,3
9,Pollution,"[air pollution and, between air pollution]",75ebad1e8641e5fe2a51214f4d97742ab59a8dae,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,[Ecologic analysis was conducted among 5 regio...,[Ecologic analysis was conducted to explore th...,None,Not yet,"Cui, Yan; Zhang, Zuo-Feng; Froines, John; Zhao...",0,3


Saving the extracted papers.

In [0]:
df_output.to_csv('pollution_papers.csv')